In [2]:
import cobra

In [3]:
model = cobra.io.read_sbml_model('../GEMS/model_compartment_cleaned.xml')

In [4]:
model.optimize()

,fluxes,reduced_costs
RXN0001_c,-298.604625,9.926868e-19
RXN0002_c,0.000000,-1.111147e-03
RXN0003_c,-1000.000000,-1.206389e-02
RXN0005_c,4.195156,0.000000e+00
RXN0006_c,0.000000,-4.656237e-03
...,...,...
EX_thrp_e,0.000000,-0.000000e+00
EX_3ump_e,0.000000,-0.000000e+00
EX_tyrp_e,0.000000,-0.000000e+00
EX_minohp_e,0.000000,0.000000e+00


In [6]:
#familiarize with model summary
print(len(model.reactions))

1773


In [10]:
import carveme
dir(carveme)
carveme.__version__

ModuleNotFoundError: No module named 'carveme'

# OPTIMIZING MODEL


In [5]:
from cobra.io import load_model
from cobra.flux_analysis import gapfill
model = cobra.io.read_sbml_model('../GEMs/model_compartment_cleaned.xml')
model.optimize()

,fluxes,reduced_costs
RXN0001_c,-298.604625,9.926868e-19
RXN0002_c,0.000000,-1.111147e-03
RXN0003_c,-1000.000000,-1.206389e-02
RXN0005_c,4.195156,0.000000e+00
RXN0006_c,0.000000,-4.656237e-03
...,...,...
EX_thrp_e,0.000000,-0.000000e+00
EX_3ump_e,0.000000,-0.000000e+00
EX_tyrp_e,0.000000,-0.000000e+00
EX_minohp_e,0.000000,0.000000e+00


In [12]:
def get_metabolite_from_exchange_rxn(model, exchange_rxn_id):
    met = list(model.reactions.get_by_id(exchange_rxn_id).metabolites)[0]
    return met

def test_get_metabolite_from_exchange_rxn():
    model = cobra.io.read_sbml_model('../GEMs/model_compartment_cleaned.xml')
    met = get_metabolite_from_exchange_rxn(model, 'EX_cpd00051_e')
    assert met.id == 'arg-L_e'

test_get_metabolite_from_exchange_rxn()

In [13]:
import pandas as pd

def create_media_db(model, medium_name, medium_description):
    """creates a media database for a model.
    The library must be a tab-separated file with four columns:

medium: short id to be passed in command line (example: X)

description: description of the medium (optional, example: Our magic X formula)

compound: compound id (example: glc). Note: the id must be the BiGG id without the compartment suffix.
    id.rsplit('_', 1)[0] was included to remove compartment suffix.

name: compound name (optional, example: Glucose)

Please note that, at this moment, CarveMe only supports metabolite ids from the BiGG database.

    Args: model (cobra.Model): a model
    medium_name (str): the name of the medium
    medium_description (str): the description of the medium
    Returns: a media database
    
    """
    media_db = pd.DataFrame(columns=['medium', 'description', 'compound', 'name'])
    #id.rsplit('_', 1)[0] was included to remove compartment suffix.
    media_db['compound'] = pd.Series([get_metabolite_from_exchange_rxn(model, rxn).id.rsplit('_', 1)[0] for rxn in model.medium])
    media_db['name'] = pd.Series([get_metabolite_from_exchange_rxn(model, rxn).name for rxn in model.medium])
    media_db['medium'] = medium_name
    media_db['description'] = medium_description
    

    return media_db

media_db = create_media_db(model, 'Base', 'Rhodobacter sphaeroides base medium')
media_db.to_csv("../rhodobacter/data/media/default_media_db.tsv", sep='\t', index=False)


In [4]:
model.reactions.get_by_id('EX_cpd00051_e')

Reaction identifier,EX_cpd00051_e
Name,EX_L_Arginine_e0
Memory address,0x23fa04ac9e0
Stoichiometry,arg-L_e <=> L_Arginine_e <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [5]:
cpd = 'EX_cpd00051_e'
cpd.split('_')[1]

'cpd00051'

In [6]:
model.medium

{'EX_cpd00051_e': 1000.0,
 'EX_cpd00119_e': 1000.0,
 'EX_cpd00039_e': 1000.0,
 'EX_cpd00084_e': 1000.0,
 'EX_cpd00053_e': 1000.0,
 'EX_cpd00107_e': 1000.0,
 'EX_cpd00322_e': 1000.0,
 'EX_cpd00156_e': 1000.0,
 'EX_cpd00060_e': 1000.0,
 'EX_cpd00731_e': 1000.0,
 'EX_cpd00042_e': 1000.0,
 'EX_cpd00111_e': 1000.0,
 'EX_cpd00166_e': 1000.0,
 'EX_cpd00058_e': 1000.0,
 'EX_cpd00034_e': 1000.0,
 'EX_cpd00030_e': 1000.0,
 'EX_cpd10515_e': 1000.0,
 'EX_cpd00149_e': 1000.0,
 'EX_cpd00028_e': 1000.0,
 'EX_cpd00268_e': 1000.0,
 'EX_cpd00048_e': 1000.0,
 'EX_cpd11574_e': 1000.0,
 'EX_cpd00118_e': 1000.0,
 'EX_cpd00264_e': 1000.0,
 'EX_cpd00540_e': 1000.0,
 'EX_cpd00129_e': 1000.0,
 'EX_cpd00098_e': 1000.0,
 'EX_cpd00179_e': 1000.0,
 'EX_cpd00588_e': 1000.0,
 'EX_cpd00314_e': 1000.0,
 'EX_cpd00100_e': 1000.0,
 'EX_cpd00105_e': 1000.0,
 'EX_cpd00154_e': 1000.0,
 'EX_cpd00082_e': 1000.0,
 'EX_cpd00009_e': 1000.0,
 'EX_cpd00023_e': 1000.0,
 'EX_cpd00041_e': 1000.0,
 'EX_cpd00132_e': 1000.0,
 'EX_cpd0000

In [6]:
model.metabolites.f6p_c.reactions

frozenset({<Reaction RXN0046_c at 0x24712f66450>,
           <Reaction RXN0343_c at 0x24713053bf0>,
           <Reaction RXN0345_c at 0x24713053e90>,
           <Reaction RXN0415_c at 0x2471222e0c0>,
           <Reaction RXN0416_c at 0x247122bf530>,
           <Reaction RXN0553_c at 0x24713181b20>,
           <Reaction RXN0556_c at 0x2471312e930>,
           <Reaction RXN0560_c at 0x24713183110>})

# Cobrapy Gapfill below
You should have three differnt models:
1. original model
2. a model with the missing metabolite (gapped)
3. Run gapfill to find one/more solutions
4. gapfill with one solution
5. Test to make sure the gapfilled working model works (model.optimize)
6. Save model with metabolite gap filled 

asssertions to check: 
1. assert original model grows
2. assert that gapped doesn't grow (assert < threshold)
3. assert that gapfilled does grow
growth = model.optimize().objective_value
threshold = 0.2
assert growth > threshold, f"Growth {growth} was not greater than threshold {threshold}"

In [7]:
import cobra
universal = cobra.Model("universal_reactions") #grabs universal model from cobra
for i in [i.id for i in model.metabolites.f6p_c.reactions]: #iterates through all reactions that f6p is involved in F6p metabolites
    reaction = model.reactions.get_by_id(i)
    universal.add_reactions([reaction.copy()])
    model.remove_reactions([reaction]) #adds f6p reactions to universal model and removes them from our working model
cobra.io.write_sbml_model(universal, "../GEMs/f6p_reactions.xml") #universal model with f6p
cobra.io.write_sbml_model(model, "../GEMs/model_with_f6p_gap.xml") #model without f6p so should not grow

In [10]:
#test to see if model grows
model_with_f6p_gap = cobra.io.read_sbml_model("../GEMs/model_with_f6p_gap.xml")
model_with_f6p_gap.optimize()

,fluxes,reduced_costs
RXN0001_c,0.0,0.0
RXN0002_c,0.0,0.0
RXN0003_c,0.0,-0.0
RXN0005_c,0.0,0.0
RXN0006_c,0.0,0.0
...,...,...
EX_thrp_e,0.0,0.0
EX_3ump_e,0.0,-0.0
EX_tyrp_e,0.0,0.0
EX_minohp_e,0.0,0.0


In [13]:
#run gapfill to replace removed metabolite into working model: view reactions to add
solutions = gapfill(model_with_f6p_gap, universal, demand_reactions=False)
for reaction in solutions[0]:
    print(reaction.id)

RXN0415_c
RXN0553_c


In [14]:
#run gapfill to replace removed metabolite into working model: take first element of solutions and add
f6p_gapfilled = model_with_f6p_gap.copy()
f6p_gapfilled.add_reactions(solutions[0])
cobra.io.write_sbml_model(f6p_gapfilled, "../GEMs/f6p_gapfilled.xml")

In [ ]:
#optimize gapfilled model to make sure it grows
f6p_gapfilled.optimize()
#assert to optimize.solutions() compared to final model

,fluxes,reduced_costs
RXN0001_c,-418.913387,-1.489030e-18
RXN0002_c,0.000000,-1.602094e-03
RXN0003_c,-1000.000000,-1.094764e-02
RXN0005_c,4.052938,0.000000e+00
RXN0006_c,0.000000,-1.789005e-02
...,...,...
EX_tyrp_e,0.000000,1.999411e-18
EX_minohp_e,0.000000,0.000000e+00
EX_etoh_e,0.000000,-0.000000e+00
RXN0415_c,5.873753,0.000000e+00


## Run CarveMe gapfill

```bash
usage: gapfill [-h] -m MEDIA [--mediadb MEDIADB] [--spent-medium SPECIES]
               [-u UNIVERSE | --universe-file UNIVERSE_FILE] [-o OUTPUT] [-v]
               [--cobra | --fbc2]
               INPUTFILE

GapFill a metabolic model for a given set of media

positional arguments:
  INPUTFILE             SBML input file

options:
  -h, --help            show this help message and exit
  -m MEDIA, --media MEDIA
                        List of media (comma-separated)
  --mediadb MEDIADB     Media database file
  --spent-medium SPECIES
                        Add spent medium compounds generated from given species (SBML   
                        model).
  -u UNIVERSE, --universe UNIVERSE
                        Pre-built universe model (default: bacteria)
  --universe-file UNIVERSE_FILE
                        Reaction universe file (SBML format)
  -o OUTPUT, --output OUTPUT
                        SBML output file
  -v, --verbose         Switch to verbose mode
  --cobra               Input SBML in old cobra format
  --fbc2                Input SBML in sbml-fbc2 format
  ```

In [18]:
%%bash
 gapfill -m Base --mediadb ../rhodobacter/data/media/default_media_db.tsv --universe-file ../GEMs/f6p_reactions.xml -o ../GEMs/f6p_gapfilled_model.xml ../GEMs/model_with_f6p_gap.xml

C:\Users\lint730\AppData\Local\Programs\Python\Python312\Lib\site-packages\reframed\io\sbml.py:173: UserWarning: Exchange reactions were not detected.
  warn("Exchange reactions were not detected.")


Restricted license - for non-production use only - expires 2025-11-24


Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\lint730\AppData\Local\Programs\Python\Python312\Scripts\gapfill.exe\__main__.py", line 7, in <module>
  File "C:\Users\lint730\AppData\Local\Programs\Python\Python312\Lib\site-packages\carveme\cli\gapfill.py", line 118, in main
    maincall(inputfile=args.input,
  File "C:\Users\lint730\AppData\Local\Programs\Python\Python312\Lib\site-packages\carveme\cli\gapfill.py", line 64, in maincall
    multiGapFill(model, universe_model, media, media_db, max_uptake=max_uptake, inplace=True,
  File "C:\Users\lint730\AppData\Local\Programs\Python\Python312\Lib\site-packages\carveme\reconstruction\gapfilling.py", line 143, in multiGapFill
    gapFill(model, universe, constraints=constraints, min_growth=min_growth,
  File "C:\Users\lint730\AppData\Local\Programs\Python\Python312\Lib\site-packages\carveme\reconstruction\gapfilling.py", line 78, i

CalledProcessError: Command 'b' gapfill -m Base --mediadb ../rhodobacter/data/media/default_media_db.tsv --universe-file ../GEMs/f6p_reactions.xml -o ../GEMs/f6p_gapfilled_model.xml ../GEMs/model_with_f6p_gap.xml\n'' returned non-zero exit status 1.

In [9]:
model.optimize().objective_value

0.0

In [10]:
solution = gapfill(model, universal, demand_reactions=False)
for reaction in solution[0]:
    print(reaction.id)

NameError: name 'gapfill' is not defined

In [11]:
universal.reactions


[<Reaction RXN0046_c at 0x23fa00db080>,
 <Reaction RXN0343_c at 0x23fa00db050>,
 <Reaction RXN0345_c at 0x23ffdb61a90>,
 <Reaction RXN0556_c at 0x23fa0552660>,
 <Reaction RXN0560_c at 0x23fa028ca10>,
 <Reaction RXN0416_c at 0x23fa0505c70>,
 <Reaction RXN0415_c at 0x23fa05057c0>,
 <Reaction RXN0553_c at 0x23fa04ace90>]

In [12]:
for metabolite in model.metabolites:
    note = metabolite.notes
    if type(note) is str:
        metabolite.notes = {'seed.id': note}

In [13]:
cobra.io.save_json_model(model, "../GEMs/iRsp1140_opt.json")
cobra.io.write_sbml_model(model, "../rhodobacter/model_gapfilled.xml")

# reverse glyoxylate shunt (rGS)

ppc, added\
mdh: RXN0143_c0, reversible\
mtk, added\
fum: RXN0148_c0, reversible\
frd: RXN0145_c0, reversible\
mcl: RXN0689_c0, reversible\
icl, added\
acn: RXN0147_c0, reversible\
acl, added

In [4]:
def add_ppc_reaction(model):
    # H2O + Phosphoenolpyruvate + CO2 -> Orthophosphate + Oxaloacetate
    reaction = cobra.Reaction('ppc')
    reaction.name = 'phosphate:oxaloacetate carboxy-lyase'
    reaction.subsystem = 'reverse glyoxylate shunt'
    reaction.add_metabolites({
        model.metabolites.get_by_id('h2o_c0'): -1.0,
        model.metabolites.get_by_id('pep_c0'): -1.0,
        model.metabolites.get_by_id('co2_c0'): -1.0, 
        model.metabolites.get_by_id('pi_c0'): 1.0,
        model.metabolites.get_by_id('oaa_c0'): 1.0
    })
    model.add_reactions([reaction])
    return model

def add_mtk_reaction(model):
    # ATP + (S)-Malate + CoA -> ADP + Orthophosphate + L-Malyl-CoA
    # Note: existing reaction RXN0690_c0 can convert L-Malyl-CoA to (S)-Malate + CoA
    reaction = cobra.Reaction('mtk')
    reaction.name = 'malate thiokinase'
    reaction.subsystem = 'reverse glyoxylate shunt'
    reaction.add_metabolites({
        model.metabolites.get_by_id('atp_c0'): -1.0,
        model.metabolites.get_by_id('mal-L_c0'): -1.0,
        model.metabolites.get_by_id('coa_c0'): -1.0,
        model.metabolites.get_by_id('adp_c0'): 1.0,
        model.metabolites.get_by_id('pi_c0'): 1.0,
        model.metabolites.get_by_id('malylcoa_c0'): 1.0
    })
    model.add_reactions([reaction])
    return model

def add_icl_reaction(model):
    # succinate + glyoxylate <=> isocitrate (reversible)
    reaction = cobra.Reaction('icl')
    reaction.name = 'isocitrate lyase' 
    reaction.subsystem = 'reverse glyoxylate shunt'
    reaction.add_metabolites({
        model.metabolites.get_by_id('glx_c0'): -1.0,
        model.metabolites.get_by_id('succ_c0'): -1.0,
        model.metabolites.get_by_id('icit_c0'): 1.0,
    })
    model.add_reactions([reaction])
    return model

def add_acl_reaction(model):
    # ATP + Citrate + CoA -> ADP + Orthophosphate + Acetyl-CoA + Oxaloacetate
    reaction = cobra.Reaction('acl')
    reaction.name = 'acetyl-CoA:oxaloacetate C-acetyltransferase'
    reaction.subsystem = 'reverse glyoxylate shunt'
    reaction.add_metabolites({
        model.metabolites.get_by_id('atp_c0'): -1.0,
        model.metabolites.get_by_id('cit_c0'): -1.0,
        model.metabolites.get_by_id('coa_c0'): -1.0,
        model.metabolites.get_by_id('adp_c0'): 1.0,
        model.metabolites.get_by_id('pi_c0'): 1.0,
        model.metabolites.get_by_id('accoa_c0'): 1.0,
        model.metabolites.get_by_id('oaa_c0'): 1.0
    })
    model.add_reactions([reaction])
    return model

def add_rGS_reactions(model):
    model = add_ppc_reaction(model)
    model = add_mtk_reaction(model)
    model = add_icl_reaction(model)
    model = add_acl_reaction(model)
    return model

In [5]:
model_rGS = model.copy()
model_rGS = add_rGS_reactions(model_rGS)

In [6]:
model_rGS

Name,Rhodobacter_sphaeroides_2_4_1_fbamdl_17
Memory address,3064e5fc0
Number of metabolites,1344
Number of reactions,1651
Number of genes,1140
Number of groups,0
Objective expression,1.0*biomass0 - 1.0*biomass0_reverse_9c18d
Compartments,"Cytosol_0, e0, p0"


In [8]:
cobra.io.save_json_model(model_rGS, "../GEMs/iRsp1140_opt_with_rGS.json")

# non-oxidative glycolysis (NOG)

fpk, added\
xpk, added\
tkt, RXN0553_c0\
tal, RXN0560_c0\
rpi, RXN0561_c0\
rpe, RXN0559_c0

In [9]:
def add_NOG_reactions(model):
    # D-Xylulose 5-phosphate + Orthophosphate <=> Acetyl phosphate + D-Glyceraldehyde 3-phosphate + H2O
    reaction = cobra.Reaction('xpk')
    reaction.name = 'D-xylulose 5-phosphate D-glyceraldehyde-3-phosphate-lyase'
    reaction.subsystem = 'non-oxidative glycolysis'
    reaction.add_metabolites({
        model.metabolites.get_by_id('xu5p-D_c0'): -1.0,
        model.metabolites.get_by_id('pi_c0'): -1.0,
        model.metabolites.get_by_id('actp_c0'): 1.0,
        model.metabolites.get_by_id('g3p_c0'): 1.0,
        model.metabolites.get_by_id('h2o_c0'): 1.0
    })
    model.add_reactions([reaction])

    # D-Fructose 6-phosphate + Orthophosphate <=> Acetyl phosphate + D-Erythrose 4-phosphate + H2O
    reaction = cobra.Reaction('fpk')
    reaction.name = 'D-fructose-6-phosphate D-erythrose-4-phosphate-lyase'
    reaction.subsystem = 'non-oxidative glycolysis'
    reaction.add_metabolites({
        model.metabolites.get_by_id('f6p_c0'): -1.0,
        model.metabolites.get_by_id('pi_c0'): -1.0,
        model.metabolites.get_by_id('actp_c0'): 1.0,
        model.metabolites.get_by_id('e4p_c0'): 1.0,
        model.metabolites.get_by_id('h2o_c0'): 1.0
    })
    model.add_reactions([reaction])
    return model

In [10]:
model_NOG = model.copy()
model_NOG = add_NOG_reactions(model_NOG)

In [11]:
model_NOG

Name,Rhodobacter_sphaeroides_2_4_1_fbamdl_17
Memory address,147351030
Number of metabolites,1344
Number of reactions,1649
Number of genes,1140
Number of groups,0
Objective expression,1.0*biomass0 - 1.0*biomass0_reverse_9c18d
Compartments,"Cytosol_0, e0, p0"


In [12]:
cobra.io.save_json_model(model_NOG, "../GEMs/iRsp1140_opt_with_NOG.json")

# Embden-Parnas-Bifido (EP-Bifido) pathway

In [13]:
def add_gnd_reaction(model):
    # 6-Phospho-D-gluconate + NADP+ <=> D-Ribulose 5-phosphate + CO2 + NADPH + H+
    reaction = cobra.Reaction('gnd')
    reaction.name = '6-phospho-D-gluconate:NADP+ 2-oxidoreductase'
    reaction.subsystem = 'EP-Bifido pathway'
    reaction.add_metabolites({
        model.metabolites.get_by_id('6pgc_c0'): -1.0,
        model.metabolites.get_by_id('nadp_c0'): -1.0,
        model.metabolites.get_by_id('ru5p-D_c0'): 1.0,
        model.metabolites.get_by_id('co2_c0'): 1.0,
        model.metabolites.get_by_id('nadph_c0'): 1.0,
        model.metabolites.get_by_id('h_c0'): 1.0,
    })
    model.add_reactions([reaction])
    return model

def add_EP_Bifido_reactions(model):
    model = add_NOG_reactions(model)
    model = add_gnd_reaction(model)
    return model

In [14]:
model_EP_Bifido = model.copy()
model_EP_Bifido = add_EP_Bifido_reactions(model_EP_Bifido)

In [15]:
model_EP_Bifido

Name,Rhodobacter_sphaeroides_2_4_1_fbamdl_17
Memory address,3083bcbb0
Number of metabolites,1344
Number of reactions,1650
Number of genes,1140
Number of groups,0
Objective expression,1.0*biomass0 - 1.0*biomass0_reverse_9c18d
Compartments,"Cytosol_0, e0, p0"


In [16]:
cobra.io.save_json_model(model_EP_Bifido, "../GEMs/iRsp1140_opt_with_EP_Bifido.json")

# malyl-CoA-glycerate cylce (MCG)

ppc, added\
mdh: RXN0143_c0, reversible\
mtk, added\
mcl: RXN0689_c0, reversible\
gcl, added\
tsr, added\
gk, added\
eno: RXN0420_c0

In [17]:
def add_2h3oppan_c0_metabolite(model):
    m_2h3oppan_c0 = cobra.Metabolite('2h3oppan_c0', formula='C3H3O4', name='2-Hydroxy-3-oxopropanoate', compartment='c0', charge = -1) #'cpd00843'
    model.add_metabolites([m_2h3oppan_c0])
    return model

def add_gcl_reaction(model):
    # 2 Glyoxylate -> 2-Hydroxy-3-oxopropanoate + CO2
    reaction = cobra.Reaction('gcl')
    reaction.name = 'glyoxylate carboxy-lyase'
    reaction.subsystem = 'malyl-CoA-glycerate cylce'
    reaction.add_metabolites({
        model.metabolites.get_by_id('glx_c0'): -2.0,
        model.metabolites.get_by_id('2h3oppan_c0'): 1.0,
        model.metabolites.get_by_id('co2_c0'): 1.0,
    })
    model.add_reactions([reaction])
    return model

def add_tsr_reaction(model):
    # 2-Hydroxy-3-oxopropanoate + NADH + H+ -> D-Glycerate + NAD+
    reaction = cobra.Reaction('tsr')
    reaction.name = '(R)-glycerate:NAD+ oxidoreductase'
    reaction.subsystem = 'malyl-CoA-glycerate cylce'
    reaction.add_metabolites({
        model.metabolites.get_by_id('2h3oppan_c0'): -1.0,
        model.metabolites.get_by_id('nadh_c0'): -1.0,
        model.metabolites.get_by_id('h_c0'): -1.0,
        model.metabolites.get_by_id('glyc-R_c0'): 1.0,
        model.metabolites.get_by_id('nad_c0'): 1.0,
    })
    model.add_reactions([reaction])
    return model

def add_gk_reaction(model):
    # D-Glycerate + ATP -> 2-Phospho-D-glycerate + ADP
    reaction = cobra.Reaction('gk')
    reaction.name = 'ATP:(R)-glycerate 2-phosphotransferase'
    reaction.subsystem = 'EP-Bifido pathway'
    reaction.add_metabolites({
        model.metabolites.get_by_id('glyc-R_c0'): -1.0,
        model.metabolites.get_by_id('atp_c0'): -1.0,
        model.metabolites.get_by_id('2pg_c0'): 1.0,
        model.metabolites.get_by_id('adp_c0'): 1.0,
    })
    model.add_reactions([reaction])
    return model

def add_MCG_reactions(model):
    model = add_ppc_reaction(model)
    model = add_mtk_reaction(model)
    model = add_2h3oppan_c0_metabolite(model)
    model = add_gcl_reaction(model)
    model = add_tsr_reaction(model)
    model = add_gk_reaction(model)
    return model

In [18]:
model_MCG = model.copy()
model_MCG = add_MCG_reactions(model_MCG)

In [19]:
model_MCG

Name,Rhodobacter_sphaeroides_2_4_1_fbamdl_17
Memory address,308db4850
Number of metabolites,1345
Number of reactions,1652
Number of genes,1140
Number of groups,0
Objective expression,1.0*biomass0 - 1.0*biomass0_reverse_9c18d
Compartments,"Cytosol_0, e0, p0"


In [20]:
cobra.io.save_json_model(model_MCG, "../GEMs/iRsp1140_opt_with_MCG.json")